# Pen-Based Recognition of Handwritten Digits 

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from MachineLearningUtils.preprocessing_data import PreProcessingData
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, auc, classification_report
from MachineLearningUtils.k_nearest_neighbor_learning import KNNLearning

In [2]:
import warnings
warnings.filterwarnings('ignore') 

In [3]:
df = pd.read_csv('du lieu cung cap/penbased-5an-nn.csv', header=None)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
1,99,80,63,100,25,76,79,68,100,62,97,23,54,0,0,16,9
2,0,73,19,99,72,100,70,73,32,48,5,18,46,0,100,72,2
3,12,77,20,62,78,40,50,0,1,17,0,64,23,98,100,100,5
4,0,98,36,42,80,85,68,42,56,0,25,23,50,37,100,32,7


In [5]:
df.shape

(10992, 17)

In [ ]:
sns.pairplot(df)

In [ ]:
df[16].value_counts()

In [ ]:
ppd = PreProcessingData(df, 'classification')
print('Total null columns = ', ppd.get_null_column().shape[0])

In [ ]:
ppd.draw_plot(sns.boxplot)

Feature 1 và 3 có outlier

1. Dữ liệu cần dự đoán là classification
2. Dựa trên kết quả pairplot, các feature không có quan hệ tuyến tính với nhau, và các điểm dữ liệu phân bố đều
3. Dữ liệu có outlier ở feature 1 và 3
4. Các features nằm trong khoảng 0 - 100 ==> không cần scale dữ liệu
5. Dữ liệu output phân bố đều ==> không cần resampling data
6. Dữ liệu không có missing data

==> chọn thuật toán liên quan đến classification

# Tiền xử lý dữ liệu

In [ ]:
output_col = 16
inputs = ppd.data.drop([output_col], axis=1)
output = ppd.data[output_col]

In [ ]:
ppd.get_k_best_features_by_random_forest(output_col, 16)

## Yêu cầu 1: Chọn thuật toán thích hợp để build model

In [ ]:
ppd.get_best_models(output_col)

Random Forest và KNN có độ chính xác tương đối cao, nhưng do độ chính xác của Random Forest thấp hơn KNN, đồng thời tốc độ chạy cũng chậm hơn ==> chọn KNN để build thuật toán

## Áp dụng dữ liệu gốc

In [ ]:
knn_learning = KNNLearning(input_data=inputs, output_data=output, mode='classification')

### 1. Tạo X_train, X_test, y_train, y_test từ dữ liệu đã đọc và chuẩn hóa,với tỷ lệ dữ liệu test là 0.3

In [ ]:
knn_learning.train_test_split(test_size=0.3)

### 2. Chọn thuật toán xây dựng model

In [ ]:
knn_learning.get_best_params()

In [ ]:
knn_learning.best_params

In [ ]:
model = knn_learning.build_model()

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier

# _model = KNeighborsClassifier(
#     algorithm='ball_tree',
#     n_neighbors=10,
#     weights='distance')
# _model.fit(X_train, y_train)

### 3. Đánh giá model

In [ ]:
print('Độ chính xác trên tập Train: ', model.score(knn_learning.X_train, knn_learning.y_train)*100, '%')
print('Độ chính xác trên tập Test: ', model.score(knn_learning.X_test, knn_learning.y_test)*100, '%')
print('Độ chính xác trên toàn dữ liệu: ', model.score(inputs, output)*100, '%')

In [ ]:
print(classification_report(knn_learning.y_test, knn_learning.y_pred))

In [ ]:
print(confusion_matrix(knn_learning.y_test, knn_learning.y_pred))

**Summary about the model:**
1. High precision: ~0.96, High recall: ~0.97, High f1 score: ~0.97
2. Có ~30% feature có độ precision, recall, f1 score chỉ chiếm khoảng 0.92, 0.93
3. High training R^2 score: 1 and High testing score: ~0.96

=> Model tương đối phù hợp

### 4. Trực quan hóa kết quả dự đoán được từ model

In [ ]:
# chọn column để trực quan hóa dữ liệu
ppd.get_k_best_features_by_extra_tree(output_col, 3)

In [ ]:
knn_learning.plot_scatter(15, 13)
knn_learning.plot_scatter(15, 13, 14)

In [ ]:
knn_learning.draw_roc_curve()

1. Các categories nằm ở từng vị trí các nhau, có thể thấy sự tập trung dữ liệu của từng category 
2. Một số category tách biệt rõ rệt với các category khác
3. Khi biểu diễn dữ liệu trên ROC currve, đường curve nằm sát các trục của đồ thị
4. Độ reacell, predicion, f1 đều cho kết quả cao ~ 97%
5. ROC chiếm ~0.98 are ==> cao
6. Tốc độ build model dựa trên Thuật toán KNN chạy nhanh

==> Mặc dù có 2 features có outlier nhưng model được build từ KNN vẫn biểu diễn tốt các điểm dữ liệu

==> KNN phù hợp cho bộ dữ liệu

## Yêu cầu 2: Áp dụng PCA để giảm chiều dữ liệu

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

### StandardScaler

In [ ]:
std_scaler = StandardScaler()
std_scaler.fit(ppd.data)

In [ ]:
data_scaler = std_scaler.transform(ppd.data)

In [ ]:
scaler_df = pd.DataFrame(data_scaler)
scaler_df.head()

In [ ]:
input_scaler = scaler_df.drop([output_col], axis=1)

### PCA tranform

In [ ]:
pca = PCA(.95)

In [ ]:
pca.fit(input_scaler)
input_pca = pca.transform(input_scaler)

### Apply KNN algorithm

#### 1. Build model with KNN

In [ ]:
knn_learning = KNNLearning(input_data=pd.DataFrame(input_pca), output_data=output, mode='classification')

In [ ]:
knn_learning.train_test_split(test_size=0.3)

In [ ]:
knn_learning.get_best_params()

In [ ]:
knn_learning.best_params

In [ ]:
model = knn_learning.build_model()

#### 2. Đánh giá model

In [ ]:
knn_learning.X_train.shape

In [ ]:
print('Độ chính xác trên tập Train: ', model.score(knn_learning.X_train, knn_learning.y_train)*100, '%')
print('Độ chính xác trên tập Test: ', model.score(knn_learning.X_test, knn_learning.y_test)*100, '%')
print('Độ chính xác trên toàn dữ liệu: ', model.score(input_pca, output)*100, '%')

In [ ]:
print(classification_report(knn_learning.y_test, knn_learning.y_pred))

In [ ]:
knn_learning.draw_roc_curve()

Khi áp dụng PCA:
1. Số chiều dữ liệu giảm được 4 ~ 25% số lượng feature (15 --> 11) ==> tốc độ được cải thiện
2. Độ chính xác sau khi thực hiện PCA giảm khoảng ~1% (khá nhỏ)
3. ROC area giảm ~0.01 (khá nhỏ)

==> có thể áp dụng PCA để giảm, nhằm tăng tốc độ thuật toán